In [1]:
import os
import numpy as np
import random
import glob
import pandas as pd
from tqdm import tqdm_notebook
from pyquaternion import Quaternion
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box
from lyft_dataset_sdk.utils.geometry_utils import BoxVisibility, transform_matrix
from lyft_dataset_sdk.utils.kitti import KittiDB
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D

In [37]:
def read_calib_file(filepath):
        with open(filepath) as f:
            lines = f.readlines()
    
        obj = lines[2].strip().split(' ')[1:]
        P2 = np.array(obj, dtype=np.float32)
        obj = lines[3].strip().split(' ')[1:]
        P3 = np.array(obj, dtype=np.float32)
        obj = lines[4].strip().split(' ')[1:]
        R0 = np.array(obj, dtype=np.float32)
        obj = lines[5].strip().split(' ')[1:]
        Tr_velo_to_cam = np.array(obj, dtype=np.float32)

        return {'P2': P2.reshape(3, 4),
                'P3': P3.reshape(3, 4),
                'R_rect': R0.reshape(3, 3),
                'Tr_velo2cam': Tr_velo_to_cam.reshape(3, 4)}

In [2]:
# load lyft test dataset
level5data = LyftDataset(data_path='.', json_path='/home/bob/data/lyft_data/train_data', verbose=True)
valid_tokens = [x.strip() for x in open('/home/bob/data/lyft2kitti/valid.txt').readlines()]
train_df = pd.read_csv('/home/bob/data/lyft_data/train.csv')

9 category,
18 attribute,
4 visibility,
15991 instance,
8 sensor,
128 calibrated_sensor,
149072 ego_pose,
148 log,
148 scene,
18634 sample,
149072 sample_data,
539765 sample_annotation,
1 map,
Done loading in 10.2 seconds.
Reverse indexing ...
Done reverse indexing in 4.2 seconds.


In [190]:
# loda kitti pred boxes
pred_folder = '/home/bob/data/lyft2kitti/training/label_2/'

In [191]:
pred_box3ds = []
pred_box2ds = []
kitti_to_nu_lidar = Quaternion(axis=(0,0,1), angle=np.pi/2)
for sample_token in tqdm_notebook(valid_tokens):
    pred_file = os.path.join(pred_folder, "{}.txt".format(sample_token))
    calib_path = pred_file.replace('label_2','calib')
    calib = read_calib_file(calib_path)
    sample = level5data.get('sample', sample_token)
    lidar_token = sample['data']['LIDAR_TOP']
    sd_record_lid = level5data.get('sample_data', lidar_token) # lidar sample data
    cs_record_lid = level5data.get('calibrated_sensor', sd_record_lid['calibrated_sensor_token']) 
    ego_pose = level5data.get("ego_pose", sd_record_lid["ego_pose_token"])
    with open(pred_file) as f:
        for line in f:
            # Parse this line into box information.
            parsed_line = KittiDB.parse_label_line(line) # in cam corrd
            center = parsed_line["xyz_camera"]
            wlh = parsed_line["wlh"]
            yaw_camera = parsed_line["yaw_camera"]
            name = parsed_line["name"]
            score = parsed_line["score"]
            # quat_box 
            quat_box = Quaternion(axis=(0, 1, 0), angle=yaw_camera) * Quaternion(axis=(1, 0, 0), angle=np.pi / 2)
            # 1: box in camera coord
            box = Box([0.0, 0.0, 0.0], wlh, quat_box, name=name, token = sample_token)
            # 2: center definition difference
            box.translate(center + np.array([0, -wlh[2] / 2, 0]))
            # 3: transform from camera to lidar
            box.rotate(Quaternion(matrix=calib['R_rect']).inverse)
            box.translate(-calib['Tr_velo2cam'][:,3])
            box.rotate(Quaternion(matrix=calib['Tr_velo2cam'][:,:3]).inverse)
            # 4: Transform to nuScenes LIDAR coord system.
            box.rotate(kitti_to_nu_lidar)
            box.score = score
            # 5: transform from lidar to ego
            box.rotate(Quaternion(cs_record_lid['rotation']))
            box.translate(np.array(cs_record_lid['translation']))
            # 6: transform from ego to global
            box.rotate(Quaternion(ego_pose['rotation']))
            box.translate(np.array(ego_pose['translation']))
            
            ## to 3D box format
            box3d = Box3D(
                sample_token=sample_token,
                translation=list(box.center),
                size=list(box.wlh),
                rotation=list(box.orientation.elements),
                name=name,
                score=score
            )
            pred_box2ds.append(box)
            pred_box3ds.append(box3d)


In [192]:
import json
pred = [b.serialize() for b in pred_box3ds]
with open("json/yolo/pred_gt.json", "w") as f:
    json.dump(pred, f)
!python mAP_evaluation.py --pred_file "json/yolo/pred_gt.json" --gt_file "json/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.9999314881405521
('animal', 1.0)
('bicycle', 1.0)
('bus', 1.0)
('car', 1.0)
('motorcycle', 1.0)
('other_vehicle', 1.0)
('pedestrian', 0.9994519051244173)
('truck', 1.0)
